# 컴피티션 링크
- https://www.kaggle.com/t/2e45abe9f1434b59a3358365432a48bb

In [57]:
# from google.colab import drive
# drive.mount('/content/drive')

- 데이터 경로 변수

In [58]:
DATA_PATH = "C:/Users/user/Desktop/데이터분석/05 Project_Final/Data/"
DATA_PATH

'C:/Users/user/Desktop/데이터분석/05 Project_Final/Data/'

- 데이터 불러오기

In [59]:
import pandas as pd
import numpy as np

train_tr = pd.read_csv(f"{DATA_PATH}store_train_transactions.csv") # 학습용 구매기록 데이터
train_target = pd.read_csv(f"{DATA_PATH}store_train.csv") # 학습용 정답 데이터
test_tr = pd.read_csv(f"{DATA_PATH}store_test_transactions.csv") # 테스트용 구매기록 데이터
submit = pd.read_csv(f"{DATA_PATH}store_submission.csv") # 제출 양식 데이터

train_tr.shape , train_target.shape , test_tr.shape , submit.shape

((523105, 7), (14940, 2), (441196, 7), (12225, 2))

In [60]:
train_tr.isnull().sum().sum(), test_tr.isnull().sum().sum()

(0, 0)

# 특성 공학(Feature Engineering)

## 날짜 형식으로 변환

In [61]:
train_tr["구매일시"] = pd.to_datetime(train_tr["구매일시"])
test_tr["구매일시"] = pd.to_datetime(test_tr["구매일시"])

## 새로 만든 feature와 병합할 고객ID로만 이루어진 데이터프레임 생성

In [62]:
train_ft = train_target[["ID"]]
test_ft = submit[["ID"]]

train_ft.shape, test_ft.shape

((14940, 1), (12225, 1))

## 구매일시를 이용한 특성생성

In [63]:
# 2004년 5월 ~ 2005년 4월 공휴일 리스트
holiday_2004 = ["2004-05-05", "2004-05-26", "2004-06-06",
                "2004-07-17", "2004-08-15", "2004-09-27",
                "2004-09-28", "2004-09-29", "2004-10-03", "2004-12-25"]

holiday_2005 = ["2005-01-01", "2005-02-08", "2005-02-09",
                "2005-02-10", "2005-03-01", "2005-04-05"]

holidays = pd.to_datetime(holiday_2004 + holiday_2005)

In [64]:
agg_list = [
        # 컬럼명, 집계 방식
        ('내점일수', lambda x: x.dt.date.nunique()),       # 내점일수 수정 : 날짜만 선택해서 nuique
        ('구매주기', lambda x: int( (x.max() - x.min()).days / x.dt.date.nunique()) ),
        ('주말방문비율', lambda x: np.mean(x.dt.weekday>4)),
        ('평일방문비율', lambda x: np.mean(x.dt.weekday<=4)),        # 평일방문비율 추가
        ('주말방문횟수', lambda x: np.sum(x.dt.weekday>4)),         # 주말방문횟수 추가
        ('평일방문횟수', lambda x: np.sum(x.dt.weekday<=4)),        # 평일방문횟수 추가

        ('봄_구매비율', lambda x: np.mean(x.dt.month.isin([3,4,5]))),
        ('여름_구매비율', lambda x: np.mean(x.dt.month.isin([6,7,8]))),
        ('가을_구매비율', lambda x: np.mean(x.dt.month.isin([9,10,11]))),
        ('겨울_구매비율', lambda x: np.mean(x.dt.month.isin([1,2,12]))),
        ('주구매요일', lambda x: x.dt.weekday.mode()[0]),
        ("주_구매_월", lambda x: x.dt.month.mode()[0]),              # 주구매 월 추가
        ("주_구매시간대", lambda x: x.dt.hour.mode()[0]),            # 주구매 시간대 추가
        ('일별평균구매횟수', lambda x:  x.count() / x.dt.date.nunique() ),
        ('거래개월수', lambda x: x.dt.date.astype(str).str[:-3].nunique() ),
        
        ('아침_구매비율', lambda x: np.mean(x.dt.hour.isin(range(6, 12)))),       # 시간대 별 구매비율 추가
        ('점심_구매비율', lambda x: np.mean(x.dt.hour.isin(range(12, 18)))),      # 시간대 별 구매비율 추가
        ('저녁_구매비율', lambda x:np.mean(x.dt.hour.isin(range(18, 22)))),       # 시간대 별 구매비율 추가
        ('야간_구매비율', lambda x:np.mean(~x.dt.hour.isin(range(6, 22)))),       # 시간대 별 구매비율 추가

        ('아침_구매횟수', lambda x: np.sum(x.dt.hour.isin(range(6, 12)))),       # 시간대 별 구매건수 추가
        ('점심_구매횟수', lambda x: np.sum(x.dt.hour.isin(range(12, 18)))),      # 시간대 별 구매건수 추가
        ('저녁_구매횟수', lambda x:np.sum(x.dt.hour.isin(range(18, 22)))),       # 시간대 별 구매건수 추가
        ('야간_구매횟수', lambda x:np.sum(~x.dt.hour.isin(range(6, 22)))),       # 시간대 별 구매건수 추가

        ('월초_구매비율', lambda x: np.mean(x.dt.day <= 10)),                     # 월초 구매비율 추가(10일 이전)
        ('월말_구매비율', lambda x: np.mean(x.dt.day >= 20)),                     # 월말 구매비율 추가(20일 이후)
        ('월초_구매횟수', lambda x: np.sum(x.dt.day <= 10)),                     # 월초 구매건수 추가(10일 이전)
        ('월말_구매횟수', lambda x: np.sum(x.dt.day >= 20)),                     # 월말 구매건수 추가(20일 이후)

        ('웨딩성수기_구매비율', lambda x: np.mean(x.dt.month.isin([4, 5, 9, 10]))),  # 결혼 성수기 시즌 구매 비율
        ('웨딩성수기_구매횟수', lambda x: ((x.dt.month == 4) | (x.dt.month == 5) | (x.dt.month == 9) | (x.dt.month == 10)).sum()),  # 결혼 성수기 시즌 구매 횟수
        
        ('1월_구매비율', lambda x: np.mean(x.dt.month == 1)), # 1월 구매비율
        ('2월_구매비율', lambda x: np.mean(x.dt.month == 2)), # 2월 구매비율
        ('3월_구매비율', lambda x: np.mean(x.dt.month == 3)), # 3월 구매비율
        ('4월_구매비율', lambda x: np.mean(x.dt.month == 4)), # 4월 구매비율
        ('5월_구매비율', lambda x: np.mean(x.dt.month == 5)), # 5월 구매비율
        ('6월_구매비율', lambda x: np.mean(x.dt.month == 6)), # 6월 구매비율
        ('7월_구매비율', lambda x: np.mean(x.dt.month == 7)), # 7월 구매비율
        ('8월_구매비율', lambda x: np.mean(x.dt.month == 8)), # 8월 구매비율
        ('9월_구매비율', lambda x: np.mean(x.dt.month == 9)), # 9월 구매비율
        ('10월_구매비율', lambda x: np.mean(x.dt.month == 10)), # 10월 구매비율
        ('11월_구매비율', lambda x: np.mean(x.dt.month == 11)), # 11월 구매비율
        ('12월_구매비율', lambda x: np.mean(x.dt.month == 12)), # 12월 구매비율

        ('1월_구매횟수', lambda x: (x.dt.month == 1).sum()),  # 1월 구매 횟수
        ('2월_구매횟수', lambda x: (x.dt.month == 2).sum()),  # 2월 구매 횟수
        ('3월_구매횟수', lambda x: (x.dt.month == 3).sum()),  # 3월 구매 횟수
        ('4월_구매횟수', lambda x: (x.dt.month == 4).sum()),  # 4월 구매 횟수
        ('5월_구매횟수', lambda x: (x.dt.month == 5).sum()),  # 5월 구매 횟수
        ('6월_구매횟수', lambda x: (x.dt.month == 6).sum()),  # 6월 구매 횟수
        ('7월_구매횟수', lambda x: (x.dt.month == 7).sum()),  # 7월 구매 횟수
        ('8월_구매횟수', lambda x: (x.dt.month == 8).sum()),  # 8월 구매 횟수
        ('9월_구매횟수', lambda x: (x.dt.month == 9).sum()),  # 9월 구매 횟수
        ('10월_구매횟수', lambda x: (x.dt.month == 10).sum()),  # 10월 구매 횟수
        ('11월_구매횟수', lambda x: (x.dt.month == 11).sum()),  # 11월 구매 횟수
        ('12월_구매횟수', lambda x: (x.dt.month == 12).sum()),   # 12월 구매 횟수

        ('1월_방문횟수', lambda x: x[x.dt.month == 1].dt.date.nunique()),
        ('2월_방문횟수', lambda x: x[x.dt.month == 2].dt.date.nunique()),
        ('3월_방문횟수', lambda x: x[x.dt.month == 3].dt.date.nunique()),
        ('4월_방문횟수', lambda x: x[x.dt.month == 4].dt.date.nunique()),
        ('5월_방문횟수', lambda x: x[x.dt.month == 5].dt.date.nunique()),
        ('6월_방문횟수', lambda x: x[x.dt.month == 6].dt.date.nunique()),
        ('7월_방문횟수', lambda x: x[x.dt.month == 7].dt.date.nunique()),
        ('8월_방문횟수', lambda x: x[x.dt.month == 8].dt.date.nunique()),
        ('9월_방문횟수', lambda x: x[x.dt.month == 9].dt.date.nunique()),
        ('10월_방문횟수', lambda x: x[x.dt.month == 10].dt.date.nunique()),
        ('11월_방문횟수', lambda x: x[x.dt.month == 11].dt.date.nunique()),
        ('12월_방문횟수', lambda x: x[x.dt.month == 12].dt.date.nunique()),

        # ("공휴일_구매유무", lambda x: x.dt.date.isin(holidays.date).any().astype(int)),       # 공휴일 구매 유무(0과 1로 반환)
        ("공휴일_구매비율", lambda x: np.mean(x.dt.date.isin(holidays.date))),                  # 공휴일 구매 비율
        ("여름휴가_구매비율", lambda x: np.mean(x.dt.month.isin([7,8]))),                       # 방학,여름휴가(7,8월) 구매비율
        ("연말_구매비율", lambda x: np.sum(x.dt.month.isin([12,1,2]))),                        # 연말(12월~2월) 구매비율
        
        ("공휴일_구매횟수", lambda x: np.sum(x.dt.date.isin(holidays.date))),                  # 공휴일 구매 비율
        ("여름휴가_구매횟수", lambda x: np.sum(x.dt.month.isin([7,8]))),                       # 방학,여름휴가(7,8월) 구매비율
        ("연말_구매횟수", lambda x: np.sum(x.dt.month.isin([12,1,2]))),                        # 연말(12월~2월) 구매비율
    ]

tmp = train_tr.groupby('ID')["구매일시"].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left',on="ID")

tmp = test_tr.groupby('ID')["구매일시"].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left',on="ID")

train_ft.shape, test_ft.shape

((14940, 72), (12225, 72))

- 월별 총구매금액 

In [65]:
for i in range(1,13):
    train_tmp = train_tr[train_tr["구매일시"].dt.month == i].groupby("ID")["구매가격"].sum().reset_index(name=f"{i}월_총구매금액")
    train_ft = train_ft.merge(train_tmp, on= "ID", how = "left")
    train_ft[f"{i}월_총구매금액"] = train_ft[f"{i}월_총구매금액"].fillna(0)

    test_tmp = test_tr[test_tr["구매일시"].dt.month == i].groupby("ID")["구매가격"].sum().reset_index(name=f"{i}월_총구매금액")
    test_ft = test_ft.merge(test_tmp, on= "ID", how = "left")
    test_ft[f"{i}월_총구매금액"] = test_ft[f"{i}월_총구매금액"].fillna(0)

train_ft.shape, test_ft.shape

((14940, 84), (12225, 84))

- 월별 평균구매금액 

In [66]:
for i in range(1,13):
    train_tmp = train_tr[train_tr["구매일시"].dt.month == i].groupby("ID")["구매가격"].mean().reset_index(name=f"{i}월_평균구매금액")
    train_ft = train_ft.merge(train_tmp, on= "ID", how = "left")
    train_ft[f"{i}월_평균구매금액"] = train_ft[f"{i}월_평균구매금액"].fillna(0)

    test_tmp = test_tr[test_tr["구매일시"].dt.month == i].groupby("ID")["구매가격"].mean().reset_index(name=f"{i}월_평균구매금액")
    test_ft = test_ft.merge(test_tmp, on= "ID", how = "left")
    test_ft[f"{i}월_평균구매금액"] = test_ft[f"{i}월_평균구매금액"].fillna(0)

train_ft.shape, test_ft.shape

((14940, 96), (12225, 96))

- 휴면회원 여부

In [67]:
agg_list = [("최근구매일", lambda x: int( (x.max() - x.min()).days))]

sort_value = train_tr.sort_values(["ID", "구매일시"], ascending= False).drop_duplicates(subset=["ID", "구매일시"],keep='first').groupby("ID").head(2)
tmp = sort_value.groupby('ID')["구매일시"].agg(agg_list).reset_index()
tmp["휴면회원"] = (tmp["최근구매일"] > 90).astype(int)
train_ft = train_ft.merge(tmp, how= 'left', on= "ID")

sort_value = test_tr.sort_values(["ID", "구매일시"], ascending= False).drop_duplicates(subset=["ID", "구매일시"],keep='first').groupby("ID").head(2)
tmp = sort_value.groupby('ID')["구매일시"].agg(agg_list).reset_index()
tmp["휴면회원"] = (tmp["최근구매일"] > 90).astype(int)
test_ft = test_ft.merge(tmp, how= 'left', on= "ID")

train_ft.shape, test_ft.shape

((14940, 98), (12225, 98))

## 지점을 이용한 특성생성

In [68]:
agg_list = [
          ("방문지점수","nunique"),
          ('주구매지점', lambda x: x.mode()[0]),
          
          ("지점다양성_비율", lambda x: x.nunique() / len(x)), # 지점 다양성 비율 추가
    ]

tmp = train_tr.groupby('ID')["지점코드"].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left',on="ID")

tmp = test_tr.groupby('ID')["지점코드"].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left',on="ID")

train_ft.shape, test_ft.shape

((14940, 101), (12225, 101))

## 브랜드코드를 이용한 특성생성

In [69]:
agg_list = [
             ('브랜드코드_nunique', 'nunique'),

             ('브랜드다양성_비율',lambda x: x.nunique() / len(x)) # 브랜드다양성 추가

             ]

tmp = train_tr.groupby('ID')["브랜드코드"].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left',on="ID")

tmp = test_tr.groupby('ID')["브랜드코드"].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left',on="ID")

train_ft.shape, test_ft.shape

((14940, 103), (12225, 103))

## 대분류를 이용한 특성생성

In [70]:
# 새로운 대분류 범주 생성
def classification(x):
    lst_kids = ['아동_스포츠', '아동문화', '케주얼_구두_아동', '아동']
    lst_young = ["영어덜트캐쥬얼", "영캐릭터", "영플라자", "영라이브"]
    lst_foods = ["생식품파트", "생식품", "공산품", "공산품파트"]
    lst_home = ["가정용품파트", "가정용품"]
    lst_clothes = ["스포츠캐쥬얼", "패션잡화", "여성캐쥬얼","여성캐주얼", "남성정장스포츠",
                "남성의류", "여성정장", "여성의류파트", "골프_유니캐쥬얼", "잡화파트", "잡화" ]
    lst_loyal = ["명품잡화", "로얄부띠끄", "로얄부틱"]

    if x in lst_kids:
        ans = "아동"
    elif x in lst_foods:
        ans = "생식품"
    elif x in lst_home:
        ans = "가정용품" 
    elif x in lst_clothes:
        ans = "의류"      
    elif x in lst_loyal:
        ans = "명품"  
    elif x in lst_young:
        ans = "영"
    else:
        ans = "기타"
    return ans

train_tr["대분류_수정"] = train_tr["대분류"].apply(classification)
test_tr["대분류_수정"] = test_tr["대분류"].apply(classification)

train_tr.head()

,ID,구매일시,지점코드,대분류,중분류,브랜드코드,구매가격,대분류_수정
0,train_13219,2004-05-01 09:40:00,A144000,공산품파트,차류,5100,59700,생식품
1,train_5590,2004-05-01 09:40:00,A144000,잡화파트,화장잡화,5101,17000,의류
2,train_7200,2004-05-01 10:20:00,A112000,공산품,용기보증,5100,34937,생식품
3,train_3010,2004-05-01 10:30:00,A373000,아동_스포츠,아동복,5105,19000,아동
4,train_10851,2004-05-01 10:30:00,A112000,가정용품,전화기_카세트,5110,215000,가정용품


In [ ]:
agg_list = [
            ('대분류_수정_nunique', 'nunique'),                 # 대분류_수정 기준 구매횟수
            ('주구매_대분류_수정', lambda x: x.mode()[0]),      # 대분류_수정 기준 주구매

            # ('대분류_수정_아동_구매여부', lambda x: int(x.str.contains("아동").any())),              # 대분류_수정에서 아동 구매 여부(1/0)
            # ('대분류_수정_생식품_구매여부', lambda x: int(x.str.contains("생식품").any())),          # 대분류_수정에서 생식품 구매 여부(1/0)
            # ('대분류_수정_가정용품_구매여부', lambda x: int(x.str.contains("가정용품").any())),      # 대분류_수정에서 가정용품 구매 여부(1/0)
            # ('대분류_수정_의류_구매여부', lambda x: int(x.str.contains("의류").any())),              # 대분류_수정에서 의류 구매 여부(1/0)
            # ('대분류_수정_명품_구매여부', lambda x: int(x.str.contains("명품").any())),              # 대분류_수정에서 명품 구매 여부(1/0)

            ('대분류_수정_아동_구매횟수', lambda x: int(x.str.contains("아동").sum())),              # 대분류_수정에서 아동 구매횟수
            ('대분류_수정_생식품_구매횟수', lambda x: int(x.str.contains("생식품").sum())),          # 대분류_수정에서 생식품 구매횟수
            ('대분류_수정_가정용품_구매횟수', lambda x: int(x.str.contains("가정용품").sum())),      # 대분류_수정에서 가정용품 구매횟수
            ('대분류_수정_의류_구매횟수', lambda x: int(x.str.contains("의류").sum())),              # 대분류_수정에서 의류 구매횟수
            ('대분류_수정_명품_구매횟수', lambda x: int(x.str.contains("명품").sum())),              # 대분류_수정에서 명품 구매횟수

            ('대분류_수정_아동_구매비중', lambda x: int(x.str.contains("아동").mean())),              # 대분류_수정에서 아동 구매횟수
            ('대분류_수정_생식품_구매비중', lambda x: int(x.str.contains("생식품").mean())),          # 대분류_수정에서 생식품 구매횟수
            ('대분류_수정_가정용품_구매비중', lambda x: int(x.str.contains("가정용품").mean())),      # 대분류_수정에서 가정용품 구매횟수
            ('대분류_수정_의류_구매비중', lambda x: int(x.str.contains("의류").mean())),              # 대분류_수정에서 의류 구매횟수
            ('대분류_수정_명품_구매비중', lambda x: int(x.str.contains("명품").mean())),              # 대분류_수정에서 명품 구매횟수


            ]

tmp = train_tr.groupby('ID')["대분류_수정"].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, on= "ID", how='left')

tmp = test_tr.groupby('ID')["대분류_수정"].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, on= "ID", how='left')

train_ft.shape, test_ft.shape

((14940, 120), (12225, 120))

## 중분류를 이용한 특성생성

In [72]:
# pd.set_option('display.max_colwidth', None)
# pd.set_option('display.max_row', None)     # 최대 행 수 제한 해제
# pp = pd.DataFrame(train_tr.groupby("대분류")["중분류"].unique())
# print(pp)

In [73]:
# 새로운 중분류 범주 생성

kids_toy_lst = [ '아동', '문화', '완구', '유아', '문구_팬시', '비디오', '완구(문화)','문구','사무용품','팬시코너(문화)','잡화(문화)', '레코드(문화)', 
                 '스포츠용품','문구(문화)', '영창(문화)','유아용품', '유아발육기','아동단품',  ]         
kids_clothes_lst = [ '아동복', '테이프', '진케주얼', '트래디셔널', '영트랜드', '유아복', '란제리', '잡화', '교복행사','스키',
                     '캐쥬얼구두', '내의', '임대구두', '수입구두', '팬시', '드레스구두',  '피혁A행사', '아동특선', '셔츠', '단품(트래디셔널)', 
                     '단품행사',  '모자', '수입종합화장품' , '아동잡화', '유아복', '아웃도어', '신생아','단품', '수영복', '슈즈','스포츠웨어' ]
kids_etc_lst = ['용기보증', '상품군미지정']

young_toy_lst = ['영잡화', '문화', '패스트푸드', '취미소품']
young_colthes_lst = ['캐릭터캐쥬얼', '캐리어캐쥬얼', '캐릭터슈즈', '수입슈즈', '임대슈즈', '패션잡화', '행사슈즈', '구두수선', '행사핸드백', '머플러', 
                     '베이직캐주얼', '영커리어캐주얼', '트렌드캐주얼', '뉴베이직캐주얼', '영캐주얼', '진캐주얼', '패션란제리', '화장품', '스포츠캐주얼', 
                     '피혁토탈(B2)', '단품_행사1', '단품_행사2', '단품', '유니섹스캐주얼', '단품_행사', '장신구', '스포츠단품', '영캐쥬얼', '행사', '진캐쥬얼']
young_etc_lst = ['용기보증']

foods_lst = ['정육', '곡물', '생선', '야채', '청과', '냉장식품', '기타식품', '조미료', '과자', '건식품', '즉석조리', '과자류', '인스턴트식품', '면류', 
             '일반조리', '건어물', '통병조림', '차류', '냉동식품','인스탄트식품', '건강식품', '일반식품명품']
foods_drink_lst = ['음료', '주류']
foods_etc_lst = ['용기보증', '일용잡화', '가정잡화', '상품군미지정', '주방용품' ]

home_elec_lst = ['전화기_카세트', 'TV_VCR', '세탁기_냉장고', '소형취사가전','가전특정','취사소형', '카세트_전화기', 'TV_VTR','소형전기','냉장고_세탁기', 
                 '컴퓨터','카메라','오디오', '건강용품','냉난방','냉장고.세탁기', '전화기','라디오.카세트', '가스렌지', 'TV.VTR', '오디오',  '조명', '가스기기']
home_furn_lst = ['전문가구(가구)', '원목(주니어)','원목_주니어', '소형가구(가구)', '침대', '소파',  '사무용품','쇼파', '식탁', '수입_종합가구',]
home_daily_lst = [ '주방용품', '문구_팬시', '침구', '욕실용품', 'L_B침구', '크리스탈', '초도자기', '카페트','수예_인테리어소품', '브랜드침구','홈데코',
                  '식탁_소품', '식기', 'N_B침구', '수입도자기',  '수예','수예침장',  '카페트_대자리', '그라스', '장식액자','직수입침구', '수예행사' ]
home_etc_lst = ['종합_수입','해외SHOP','용기보증', '상품군미지정', '통판', '특판','정장']

clothes_lst = ['수입의류', '단품', '캐릭터', '엘레강스', '타운웨어', '트래디셔널', '타운', '칼라드래디셔널', '영캐주얼', '미씨캐릭터', '인텔리젼스캐주얼', 
               '엘레강스캐주얼', '영트랜디', '디자이너캐릭터', '캐릭터캐주얼', '진캐쥬얼', '니트웨어', '부띠끄', '디자이너부띠끄', '엘레강스부틱', '레포츠단품', 
               '니트', '수입명품', '수입의류행사', '모피니트', '레포츠', '훼미닌부틱', '마담SIZE', '모피_피혁', '부띠끄행사', 'TOP디자이너', '모피.피혁',
               '레이디숍A', '하이캐쥬얼', '타운단품', '디자이너숍', '캐릭터캐쥬얼', '수입캐주얼', '미시케쥬얼', '영캐쥬얼', '디자이너니트', '인텔리젼스', 
               '모피', '디자이너', '행사_단품', '셔츠', '남성잡화', '수입', '라이센스', '어덜트', '마춤', '트.단품', '디자이너캐주얼', '내셔날', '내셔널', 
               '특정', '정장행사', '행사구두(5F)', '로얄', '피혁', '단품_행사(캐릭터)', '골프(LC)', '트레디셔널캐주얼', '트래디셔널캐쥬얼',
               '단품_행사(트래디셔널)', '캐주얼단품', '골프웨어', '스포츠웨어', '수영복', '골프단품', '임대골프', 'DC캐주얼', '스포츠단품', '로얄수입행사', 
               '단품(트래디셔널)', '스키', '스포츠단품', '아웃도어', '골프(단품)', '골프(NB)', '캐쥬얼단품', '골프(국내)','골프(수입)' ]
clothes_beuty_lst = ['화장잡화', '수입종합화장품', '향수','색조화장품', '국내화장품', '수입향수','국산화장품']
clothes_goods_lst = ['잡화', '양말', '우산장갑', '스카프', '손수건', '수입ACC', 'NB제화', '싸롱화', '핸드백', '토탈', '국내종합화장품', '보석', '시계'
                     '핸드백행사', '가방', '패션ACC', '헤어ACC', '스타킹', '준보석', '모자', '구두행사', '선글라스', '피혁소품행사', 'NB핸드백', '머플러',
                      '란제리', '우산_장갑',  '골프용품',' 골프(용품)', '여성구두', '구두임대', '로얄부틱2F', '핸드백임대', '썬그라스','패션시계',
                      '수입피혁', '일반ACC', '임대핸드백','피혁소품','넥타이','피혁B행사', '스포츠용퓸', '수입악세사리', '지갑_벨트', '행사핸드백', 
                      '선글래스', '행사소품', '패션악세사리', '헤어악세사리','남성구두', '스포츠슈즈', '용품', '스포츠용품' ]
clothes_etc_lst = ['상품군미지정', '행사', '용기보증','상품개발지원','GBR  지원','욕실용품','특선행사' ]

loyal_lst = ['수입부띠끄', '국내부띠끄', '로얄수입행사','잡화토탈', '엘레강스부틱', '의류기타', '행사', '토탈부틱','패션시계', '색조화장품', '핸드백', 
             '란제리', '수입향수', '명품', '수입종합화장품', '로얄부틱2F', '모자', '수입액세서리', '여성구두', '구두임대', '우산_장갑', '장신구행사업체', 
             '머플러', '넥타이', '수입명품', '일반악세사리', '가방', '보석', '헤어액세사리', '임대화장품', '준보석', '스타킹', '양말', '손수건', '화장잡화',
               '수입행사', '스카프', '핸드백임대' ]
loyal_etc_lst = [ '페레  지원', '용기보증', '상품군미지정']


new_lst = []
for x in range(train_tr.shape[0]):
    if  (train_tr["대분류_수정"][x] == "아동") & (train_tr["중분류"][x] in kids_toy_lst):
        new_lst.append("아동_완구")
    elif (train_tr["대분류_수정"][x] == "아동") & (train_tr["중분류"][x] in kids_clothes_lst):
        new_lst.append("아동_의류")
    elif (train_tr["대분류_수정"][x] == "아동") & (train_tr["중분류"][x] in kids_etc_lst):
        new_lst.append("아동_기타")

    elif (train_tr["대분류_수정"][x] == "영") & (train_tr["중분류"][x] in young_toy_lst):
        new_lst.append("영_완구")     
    elif (train_tr["대분류_수정"][x] == "영") & (train_tr["중분류"][x] in young_colthes_lst):
        new_lst.append("영_의류")
    elif (train_tr["대분류_수정"][x] == "영") & (train_tr["중분류"][x] in young_etc_lst):
        new_lst.append("영_기타")

    elif (train_tr["대분류_수정"][x] == "생식품") & (train_tr["중분류"][x] in foods_lst):
        new_lst.append("식료품_식료품")
    elif (train_tr["대분류_수정"][x] == "생식품") & (train_tr["중분류"][x] in foods_drink_lst):
        new_lst.append("식료품_음료")
    elif (train_tr["대분류_수정"][x] == "생식품") & (train_tr["중분류"][x] in foods_etc_lst):
        new_lst.append("식료품_기타")

    elif (train_tr["대분류_수정"][x] == "가정용품") & (train_tr["중분류"][x] in home_elec_lst):
        new_lst.append("가정용품_가전")
    elif (train_tr["대분류_수정"][x] == "가정용품") & (train_tr["중분류"][x] in home_furn_lst):
        new_lst.append("가정용품_가구")
    elif (train_tr["대분류_수정"][x] == "가정용품") & (train_tr["중분류"][x] in home_daily_lst):
        new_lst.append("가정용품_잡화")
    elif (train_tr["대분류_수정"][x] == "가정용품") & (train_tr["중분류"][x] in home_etc_lst):
        new_lst.append("가정용품_기타")

    elif (train_tr["대분류_수정"][x] == "의류") & (train_tr["중분류"][x] in clothes_lst):
        new_lst.append("의류_의류")
    elif (train_tr["대분류_수정"][x] == "의류") & (train_tr["중분류"][x] in clothes_beuty_lst):
        new_lst.append("의류_화장품")
    elif (train_tr["대분류_수정"][x] == "의류") & (train_tr["중분류"][x] in clothes_goods_lst):
        new_lst.append("의류_잡화")
    elif (train_tr["대분류_수정"][x] == "의류") & (train_tr["중분류"][x] in clothes_etc_lst):
        new_lst.append("의류_기타")

    elif (train_tr["대분류_수정"][x] == "명품") & (train_tr["중분류"][x] in loyal_lst):
        new_lst.append("명품_명품")
    elif (train_tr["대분류_수정"][x] == "명품") & (train_tr["중분류"][x] in loyal_etc_lst):
        new_lst.append("명품_기타")
    else:
        new_lst.append("기타")

    
train_tr["중분류_수정"] = new_lst

train_tr.head()

,ID,구매일시,지점코드,대분류,중분류,브랜드코드,구매가격,대분류_수정,중분류_수정
0,train_13219,2004-05-01 09:40:00,A144000,공산품파트,차류,5100,59700,생식품,식료품_식료품
1,train_5590,2004-05-01 09:40:00,A144000,잡화파트,화장잡화,5101,17000,의류,의류_화장품
2,train_7200,2004-05-01 10:20:00,A112000,공산품,용기보증,5100,34937,생식품,식료품_기타
3,train_3010,2004-05-01 10:30:00,A373000,아동_스포츠,아동복,5105,19000,아동,아동_의류
4,train_10851,2004-05-01 10:30:00,A112000,가정용품,전화기_카세트,5110,215000,가정용품,가정용품_가전


In [74]:
new_lst = []
for x in range(test_tr.shape[0]):
    if  (test_tr["대분류_수정"][x] == "아동") & (test_tr["중분류"][x] in kids_toy_lst):
        new_lst.append("아동_완구")
    elif (test_tr["대분류_수정"][x] == "아동") & (test_tr["중분류"][x] in kids_clothes_lst):
        new_lst.append("아동_의류")
    elif (test_tr["대분류_수정"][x] == "아동") & (test_tr["중분류"][x] in kids_etc_lst):
        new_lst.append("아동_기타")

    elif (test_tr["대분류_수정"][x] == "영") & (test_tr["중분류"][x] in young_toy_lst):
        new_lst.append("영_완구")     
    elif (test_tr["대분류_수정"][x] == "영") & (test_tr["중분류"][x] in young_colthes_lst):
        new_lst.append("영_의류")
    elif (test_tr["대분류_수정"][x] == "영") & (test_tr["중분류"][x] in young_etc_lst):
        new_lst.append("영_기타")

    elif (test_tr["대분류_수정"][x] == "생식품") & (test_tr["중분류"][x] in foods_lst):
        new_lst.append("식료품_식료품")
    elif (test_tr["대분류_수정"][x] == "생식품") & (test_tr["중분류"][x] in foods_drink_lst):
        new_lst.append("식료품_음료")
    elif (test_tr["대분류_수정"][x] == "생식품") & (test_tr["중분류"][x] in foods_etc_lst):
        new_lst.append("식료품_기타")

    elif (test_tr["대분류_수정"][x] == "가정용품") & (test_tr["중분류"][x] in home_elec_lst):
        new_lst.append("가정용품_가전")
    elif (test_tr["대분류_수정"][x] == "가정용품") & (test_tr["중분류"][x] in home_furn_lst):
        new_lst.append("가정용품_가구")
    elif (test_tr["대분류_수정"][x] == "가정용품") & (test_tr["중분류"][x] in home_daily_lst):
        new_lst.append("가정용품_잡화")
    elif (test_tr["대분류_수정"][x] == "가정용품") & (test_tr["중분류"][x] in home_etc_lst):
        new_lst.append("가정용품_기타")

    elif (test_tr["대분류_수정"][x] == "의류") & (test_tr["중분류"][x] in clothes_lst):
        new_lst.append("의류_의류")
    elif (test_tr["대분류_수정"][x] == "의류") & (test_tr["중분류"][x] in clothes_beuty_lst):
        new_lst.append("의류_화장품")
    elif (test_tr["대분류_수정"][x] == "의류") & (test_tr["중분류"][x] in clothes_goods_lst):
        new_lst.append("의류_잡화")
    elif (test_tr["대분류_수정"][x] == "의류") & (test_tr["중분류"][x] in clothes_etc_lst):
        new_lst.append("의류_기타")

    elif (test_tr["대분류_수정"][x] == "명품") & (test_tr["중분류"][x] in loyal_lst):
        new_lst.append("명품_명품")
    elif (test_tr["대분류_수정"][x] == "명품") & (test_tr["중분류"][x] in loyal_etc_lst):
        new_lst.append("명품_기타")
    else:
        new_lst.append("기타")

    
test_tr["중분류_수정"] = new_lst

test_tr.head()

,ID,구매일시,지점코드,대분류,중분류,브랜드코드,구매가격,대분류_수정,중분류_수정
0,test_3366,2004-05-01 10:20:00,A373000,생식품,건식품,5100,7000,생식품,식료품_식료품
1,test_9389,2004-05-01 10:23:00,A373000,명품잡화,스타킹,5104,46000,명품,명품_명품
2,test_8190,2004-05-01 10:30:00,A144000,잡화파트,수입종합화장품,5106,260000,의류,의류_화장품
3,test_8835,2004-05-01 10:30:00,A144000,잡화파트,준보석,5107,17000,의류,의류_잡화
4,test_1167,2004-05-01 10:30:00,A373000,남성의류,셔츠,5108,138000,의류,의류_의류


In [75]:
agg_list = [
            ('중분류_수정_nunique', 'nunique'),
            ('주구매_중분류_수정', lambda x: x.mode()[0]),

            ]
tmp = train_tr.groupby('ID')["중분류_수정"].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, on= "ID", how='left')

tmp = test_tr.groupby('ID')["중분류_수정"].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, on= "ID", how='left')

train_ft.shape, test_ft.shape

((14940, 122), (12225, 122))

## 구매가격을 이용한 특성생성

In [77]:
agg_list = [
        ('총구매액',lambda x: x[x > 0].sum()),         # 0 이상에서 적용
        ('구매횟수', lambda x: x[x > 0].count()),      # 0 이상에서 적용
        ('평균구매액', lambda x: x[x > 0].mean()),     # 0 이상에서 적용
        ('최대구매액', 'max'),
        ('최소구매액',lambda x: x[x > 0].min() ) ,
        ('환불금액',lambda x: x[x < 0].sum() ) ,
        ('환불건수', lambda x: ( x < 0 ).sum() ),
        ('구매금액표준편차',lambda x: x[x > 0].std() ),

        # ('평균저가구매비율', lambda x: (x <= 102479.91701049241).sum() / len(x)), # 저가 구매 비율 (구매 총평균에 비례)
        # ('평균고가구매비율', lambda x: (x > 102479.91701049241).sum() / len(x)),   # 고가 구매 비율 (구매 총평균에 비례)

        ('저가_구매횟수', lambda x: ((x > 0) & (x <= 28000)).sum()),            # train_tr의 사분위수 기준
        ('중저가_구매횟수', lambda x: ((x > 28000) & (x <= 58000)).sum()),
        ('중고가_구매횟수', lambda x: ((x > 58000) & (x <= 128000)).sum()),
        ('고가_구매횟수', lambda x: ((x > 128000) & (x <= 30820000)).sum()),

        ('저가_구매비율', lambda x: ((x > 0) & (x <= 28000)).mean()), 
        ('중저가_구매비율', lambda x: ((x > 28000) & (x <= 58000)).mean()),
        ('중고가_구매비율', lambda x: ((x > 58000) & (x <= 128000)).mean()),
        ('고가_구매비율', lambda x: ((x > 128000) & (x <= 30820000)).mean()),
        
        ("공휴일_총구매금액", lambda x: np.sum(x[train_tr["구매일시"].dt.date.isin(holidays.date)])),       # 공휴일 총구매금액
        ("여름휴가_총구매금액", lambda x: np.sum(x[train_tr["구매일시"].dt.month.isin([7,8])])),             # 방학,여름휴가(7,8월) 총구매금액
        ("연말_총구매금액", lambda x: np.sum(x[train_tr["구매일시"].dt.month.isin([12,1,2])])),              # 연말(12월~2월) 총구매금액
    ]
    

tmp = train_tr.groupby('ID')["구매가격"].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, on= "ID", how='left')
train_ft["구매금액표준편차"] = train_ft["구매금액표준편차"].fillna(0)

tmp = test_tr.groupby('ID')["구매가격"].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, on= "ID", how='left')
test_ft["구매금액표준편차"] = test_ft["구매금액표준편차"].fillna(0)

train_ft.shape, test_ft.shape

((14940, 141), (12225, 141))

- 등급 범주 생성

In [78]:
def grade(x):
    if x >= 10000000:
        ans = 1
    elif 5000000 <= x & x < 10000000:
        ans = 2
    elif 1000000 <= x & x < 5000000:
        ans = 3
    elif 0 <= x & x < 1000000:
        ans = 4
    return ans

train_ft["등급"] = train_ft["총구매액"].apply(grade)
test_ft["등급"] = test_ft["총구매액"].apply(grade)

train_ft.shape, test_ft.shape

((14940, 142), (12225, 142))

- 지점별 금액 합계

In [79]:
# 지점별 합계 금액 코드 추가 (train)
a144000_sum = (train_tr[train_tr["지점코드"] == "A144000"].groupby('ID')['구매가격'].sum().reset_index(name='A144000지점_구매금액'))
train_ft = train_ft.merge(a144000_sum, how='left', on='ID')

a112000_sum = (train_tr[train_tr["지점코드"] == "A112000"].groupby('ID')['구매가격'].sum().reset_index(name='A112000지점_구매금액'))
train_ft = train_ft.merge(a112000_sum, how='left', on='ID')

a373000_sum = (train_tr[train_tr["지점코드"] == "A373000"].groupby('ID')['구매가격'].sum().reset_index(name='A373000지점_구매금액'))
train_ft = train_ft.merge(a373000_sum, how='left', on='ID')

a202000_sum = (train_tr[train_tr["지점코드"] == "A202000"].groupby('ID')['구매가격'].sum().reset_index(name='A202000지점_구매금액'))
train_ft = train_ft.merge(a202000_sum, how='left', on='ID')

#지점별 합계 금액 코드 추가 (test)
a144000_sum = (test_tr[test_tr["지점코드"] == "A144000"].groupby('ID')['구매가격'].sum().reset_index(name='A144000지점_구매금액'))
test_ft = test_ft.merge(a144000_sum, how='left', on='ID')

a112000_sum = (test_tr[test_tr["지점코드"] == "A112000"].groupby('ID')['구매가격'].sum().reset_index(name='A112000지점_구매금액'))
test_ft = test_ft.merge(a112000_sum, how='left', on='ID')

a373000_sum = (test_tr[train_tr["지점코드"] == "A373000"].groupby('ID')['구매가격'].sum().reset_index(name='A373000지점_구매금액'))
test_ft = test_ft.merge(a373000_sum, how='left', on='ID')

a202000_sum = (test_tr[train_tr["지점코드"] == "A202000"].groupby('ID')['구매가격'].sum().reset_index(name='A202000지점_구매금액'))
test_ft = test_ft.merge(a202000_sum, how='left', on='ID')

train_ft["A144000지점_구매금액"] = train_ft["A144000지점_구매금액"].fillna(0)
train_ft["A112000지점_구매금액"] = train_ft["A112000지점_구매금액"].fillna(0)
train_ft["A373000지점_구매금액"] = train_ft["A373000지점_구매금액"].fillna(0)
train_ft["A202000지점_구매금액"] = train_ft["A202000지점_구매금액"].fillna(0)

test_ft["A144000지점_구매금액"] = test_ft["A144000지점_구매금액"].fillna(0)
test_ft["A112000지점_구매금액"] = test_ft["A112000지점_구매금액"].fillna(0)
test_ft["A373000지점_구매금액"] = test_ft["A373000지점_구매금액"].fillna(0)
test_ft["A202000지점_구매금액"] = test_ft["A202000지점_구매금액"].fillna(0)

train_ft.shape, test_ft.shape

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_8860\3346531936.py:21: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  a373000_sum = (test_tr[train_tr["지점코드"] == "A373000"].groupby('ID')['구매가격'].sum().reset_index(name='A373000지점_구매금액'))
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_8860\3346531936.py:24: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  a202000_sum = (test_tr[train_tr["지점코드"] == "A202000"].groupby('ID')['구매가격'].sum().reset_index(name='A202000지점_구매금액'))


((14940, 146), (12225, 146))

- 지점별 구매 브랜드 갯수

In [80]:
#지점별 구매 브랜드 갯수 추가(train)
a144000_brand_counts = ( train_tr[train_tr['지점코드'] == 'A144000'].groupby('ID')['브랜드코드'].nunique().reset_index(name='A1440000_구매브랜드_갯수'))
train_ft = train_ft.merge(a144000_brand_counts, how='left', on='ID')

a112000_brand_counts = ( train_tr[train_tr['지점코드'] == 'A112000'].groupby('ID')['브랜드코드'].nunique().reset_index(name='A112000_구매브랜드_갯수'))
train_ft = train_ft.merge(a112000_brand_counts, how='left', on='ID')

a373000_brand_counts = ( train_tr[train_tr['지점코드'] == 'A373000'].groupby('ID')['브랜드코드'].nunique().reset_index(name='A373000_구매브랜드_갯수'))
train_ft = train_ft.merge(a373000_brand_counts, how='left', on='ID')

a202000_brand_counts = ( train_tr[train_tr['지점코드'] == 'A202000'].groupby('ID')['브랜드코드'].nunique().reset_index(name='A202000_구매브랜드_갯수'))
train_ft = train_ft.merge(a202000_brand_counts, how='left', on='ID')
train_ft.head()

#지점별 구매 브랜드 갯수 추가(test)
a144000_brand_counts = ( test_tr[test_tr['지점코드'] == 'A144000'].groupby('ID')['브랜드코드'].nunique().reset_index(name='A1440000_구매브랜드_갯수'))
test_ft = test_ft.merge(a144000_brand_counts, how='left', on='ID')

a112000_brand_counts = ( test_tr[test_tr['지점코드'] == 'A112000'].groupby('ID')['브랜드코드'].nunique().reset_index(name='A112000_구매브랜드_갯수'))
test_ft = test_ft.merge(a112000_brand_counts, how='left', on='ID')

a373000_brand_counts = ( test_tr[test_tr['지점코드'] == 'A373000'].groupby('ID')['브랜드코드'].nunique().reset_index(name='A373000_구매브랜드_갯수'))
test_ft = test_ft.merge(a373000_brand_counts, how='left', on='ID')

a202000_brand_counts = ( test_tr[test_tr['지점코드'] == 'A202000'].groupby('ID')['브랜드코드'].nunique().reset_index(name='A202000_구매브랜드_갯수'))
test_ft = test_ft.merge(a202000_brand_counts, how='left', on='ID')


train_ft["A1440000_구매브랜드_갯수"] = train_ft["A1440000_구매브랜드_갯수"].fillna(0)
train_ft["A112000_구매브랜드_갯수"] = train_ft["A112000_구매브랜드_갯수"].fillna(0)
train_ft["A373000_구매브랜드_갯수"] = train_ft["A373000_구매브랜드_갯수"].fillna(0)
train_ft["A202000_구매브랜드_갯수"] = train_ft["A202000_구매브랜드_갯수"].fillna(0)

test_ft["A1440000_구매브랜드_갯수"] = test_ft["A1440000_구매브랜드_갯수"].fillna(0)
test_ft["A112000_구매브랜드_갯수"] = test_ft["A112000_구매브랜드_갯수"].fillna(0)
test_ft["A373000_구매브랜드_갯수"] = test_ft["A373000_구매브랜드_갯수"].fillna(0)
test_ft["A202000_구매브랜드_갯수"] = test_ft["A202000_구매브랜드_갯수"].fillna(0)


train_ft.shape, test_ft.shape

((14940, 150), (12225, 150))

## pivot_table을 이용한 특성 생성

- 대분류_수정 기준 금액 합계


In [81]:
train_tmp = pd.pivot_table(train_tr,index="ID",columns="대분류_수정",values="구매가격",aggfunc="sum",fill_value=0)\
            .add_prefix("pv_대분류수정_").add_suffix("_총구매금액").reset_index()
train_ft = train_ft.merge(train_tmp,how="left",on="ID")

test_tmp = pd.pivot_table(test_tr,index="ID",columns="대분류_수정",values="구매가격",aggfunc="sum",fill_value=0)\
            .add_prefix("pv_대분류수정_").add_suffix("_총구매금액").reset_index()

for col in train_tmp.columns:
    if col not in test_tmp.columns:
        test_tmp[col] = 0

test_tmp = test_tmp[train_tmp.columns]
test_ft = test_ft.merge(test_tmp,how="left",on="ID")

train_ft.shape, test_ft.shape

((14940, 156), (12225, 156))

- 대분류_수정 기준 구매횟수


In [82]:
train_tmp = pd.pivot_table(train_tr,index="ID",columns="대분류_수정",values="구매가격",aggfunc="count",fill_value=0)\
            .add_prefix("pv_대분류수정_").add_suffix("_구매횟수").reset_index()
train_ft = train_ft.merge(train_tmp,how="left",on="ID")

test_tmp = pd.pivot_table(test_tr,index="ID",columns="대분류_수정",values="구매가격",aggfunc="count",fill_value=0)\
            .add_prefix("pv_대분류수정_").add_suffix("_구매횟수").reset_index()

for col in train_tmp.columns:
    if col not in test_tmp.columns:
        test_tmp[col] = 0

test_tmp = test_tmp[train_tmp.columns]
test_ft = test_ft.merge(test_tmp,how="left",on="ID")

train_ft.shape, test_ft.shape

((14940, 162), (12225, 162))

- 대분류_수정 기준 구입비중


In [83]:
for col in train_tr["대분류_수정"].unique():
    train_ft[f"pv_대분류수정_{col}_구입비중"] = train_ft[f"pv_대분류수정_{col}_총구매금액"]/train_ft["총구매액"]
    test_ft[f"pv_대분류수정_{col}_구입비중"] = test_ft[f"pv_대분류수정_{col}_총구매금액"]/test_ft["총구매액"]

train_ft.shape, test_ft.shape

((14940, 168), (12225, 168))

- 대분류별 구매주기

In [84]:
train_tmp = pd.pivot_table(train_tr,index="ID",columns="대분류_수정",values="구매일시",\
                           aggfunc=(lambda x: int( (x.max() - x.min()).days / x.dt.date.nunique())), fill_value=0)\
                           .add_prefix("대분류수정_").add_suffix("_구매주기").reset_index()
train_ft = train_ft.merge(train_tmp, on="ID", how= "left")

test_tmp = pd.pivot_table(test_tr,index="ID",columns="대분류_수정",values="구매일시",\
                           aggfunc=(lambda x: int( (x.max() - x.min()).days / x.dt.date.nunique())), fill_value=0)\
                           .add_prefix("대분류수정_").add_suffix("_구매주기").reset_index()
test_ft = test_ft.merge(test_tmp, on="ID", how= "left")

train_ft.shape, test_ft.shape

((14940, 174), (12225, 174))

- 중분류_수정 기준 금액 합계


In [85]:
train_tmp = pd.pivot_table(train_tr,index="ID",columns="중분류_수정",values="구매가격",aggfunc="sum",fill_value=0)\
            .add_prefix("pv_중분류수정_").add_suffix("_총구매금액").reset_index()
train_ft = train_ft.merge(train_tmp,how="left",on="ID")

test_tmp = pd.pivot_table(test_tr,index="ID",columns="중분류_수정",values="구매가격",aggfunc="sum",fill_value=0)\
            .add_prefix("pv_중분류수정_").add_suffix("_총구매금액").reset_index()

for col in train_tmp.columns:
    if col not in test_tmp.columns:
        test_tmp[col] = 0

test_tmp = test_tmp[train_tmp.columns]
test_ft = test_ft.merge(test_tmp,how="left",on="ID")

train_ft.shape, test_ft.shape

((14940, 194), (12225, 194))

- 중분류_수정 기준 구매횟수


In [86]:
train_tmp = pd.pivot_table(train_tr,index="ID",columns="중분류_수정",values="구매가격",aggfunc="count",fill_value=0)\
            .add_prefix("pv_중분류수정_").add_suffix("_구매횟수").reset_index()
train_ft = train_ft.merge(train_tmp,how="left",on="ID")

test_tmp = pd.pivot_table(test_tr,index="ID",columns="중분류_수정",values="구매가격",aggfunc="count",fill_value=0)\
            .add_prefix("pv_중분류수정_").add_suffix("_구매횟수").reset_index()

for col in train_tmp.columns:
    if col not in test_tmp.columns:
        test_tmp[col] = 0

test_tmp = test_tmp[train_tmp.columns]
test_ft = test_ft.merge(test_tmp,how="left",on="ID")

train_ft.shape, test_ft.shape

((14940, 214), (12225, 214))

- 중분류_수정 기준 구입비중


In [87]:
for col in train_tr["중분류_수정"].unique():
    train_ft[f"pv_중분류수정_{col}_구입비중"] = train_ft[f"pv_중분류수정_{col}_총구매금액"]/train_ft["총구매액"]
    test_ft[f"pv_중분류수정_{col}_구입비중"] = test_ft[f"pv_중분류수정_{col}_총구매금액"]/test_ft["총구매액"]

train_ft.shape, test_ft.shape

((14940, 234), (12225, 234))

- 중분류별 구매주기

In [88]:
train_tmp = pd.pivot_table(train_tr,index="ID",columns="중분류_수정",values="구매일시",\
                           aggfunc=(lambda x: int( (x.max() - x.min()).days / x.dt.date.nunique())), fill_value=0)\
                           .add_prefix("중분류수정_").add_suffix("_구매주기").reset_index()
train_ft = train_ft.merge(train_tmp, on="ID", how= "left")

test_tmp = pd.pivot_table(test_tr,index="ID",columns="중분류_수정",values="구매일시",\
                           aggfunc=(lambda x: int( (x.max() - x.min()).days / x.dt.date.nunique())), fill_value=0)\
                           .add_prefix("중분류수정_").add_suffix("_구매주기").reset_index()
test_ft = test_ft.merge(test_tmp, on="ID", how= "left")

train_ft.shape, test_ft.shape

((14940, 254), (12225, 254))

- 공휴일에 구매한 대분류별 가격

In [89]:
# 공휴일에 구매한 대분류별 가격 - train
holiday_data = train_tr[train_tr["구매일시"].dt.date.isin(holidays.date)]

holiday_pivot = pd.pivot_table(holiday_data, index= "ID", columns= "대분류_수정", values= "구매가격", aggfunc="sum", fill_value= 0)\
                .add_prefix("공휴일_대분류수정_").add_suffix("_총구매금액").reset_index()

train_ft = train_ft.merge(holiday_pivot, how='left', on="ID")

for col in holiday_pivot.columns:
    if col == "ID":
        pass
    else: 
        train_ft[col] = train_ft[col].fillna(0)

# 공휴일에 구매한 대분류별 가격 - test
holiday_data = test_tr[test_tr["구매일시"].dt.date.isin(holidays.date)]

holiday_pivot = pd.pivot_table(holiday_data, index= "ID", columns= "대분류_수정", values= "구매가격", aggfunc="sum", fill_value= 0)\
                .add_prefix("공휴일_대분류수정_").add_suffix("_총구매금액").reset_index()

test_ft = test_ft.merge(holiday_pivot, how='left', on="ID")

for col in holiday_pivot.columns:
    if col == "ID":
        pass
    else: 
        test_ft[col] = test_ft[col].fillna(0)

train_ft.shape, test_ft.shape

((14940, 260), (12225, 260))

- 공휴일에 구매한 대분류별 횟수

In [90]:
# 공휴일에 구매한 대분류별 횟수 - train
holiday_data = train_tr[train_tr["구매일시"].dt.date.isin(holidays.date)]

holiday_pivot = pd.pivot_table(holiday_data, index= "ID", columns= "대분류_수정", values= "구매가격", aggfunc="count", fill_value= 0)\
                .add_prefix("공휴일_대분류수정_").add_suffix("_구매횟수").reset_index()

train_ft = train_ft.merge(holiday_pivot, how='left', on="ID")

for col in holiday_pivot.columns:
    if col == "ID":
        pass
    else: 
        train_ft[col] = train_ft[col].fillna(0)

# 공휴일에 구매한 대분류별 가격 - test
holiday_data = test_tr[test_tr["구매일시"].dt.date.isin(holidays.date)]

holiday_pivot = pd.pivot_table(holiday_data, index= "ID", columns= "대분류_수정", values= "구매가격", aggfunc="count", fill_value= 0)\
                .add_prefix("공휴일_대분류수정_").add_suffix("_구매횟수").reset_index()

test_ft = test_ft.merge(holiday_pivot, how='left', on="ID")

for col in holiday_pivot.columns:
    if col == "ID":
        pass
    else: 
        test_ft[col] = test_ft[col].fillna(0)

train_ft.shape, test_ft.shape

((14940, 266), (12225, 266))

- 대분류별 최대평균금액  

In [91]:
train_tmp = train_tr.pivot_table(index= ["ID", "대분류_수정"], values= "구매가격", aggfunc= "mean")\
      .sort_values(by=['ID', "구매가격"], ascending=False).groupby('ID').head(1).reset_index()
train_tmp = train_tmp.rename(columns= {"대분류_수정": "대분류_수정_평균금액최대", "구매가격" : "대분류_수정_평균구매가격"})
train_ft = train_ft.merge(train_tmp, on= "ID", how= "left")

test_tmp = test_tr.pivot_table(index= ["ID", "대분류_수정"], values= "구매가격", aggfunc= "mean")\
      .sort_values(by=['ID', "구매가격"], ascending=False).groupby('ID').head(1).reset_index()
test_tmp = test_tmp.rename(columns= {"대분류_수정": "대분류_수정_평균금액최대", "구매가격" : "대분류_수정_평균구매가격"})
test_ft = test_ft.merge(test_tmp, on= "ID", how= "left")


train_ft.shape, test_ft.shape

((14940, 268), (12225, 268))

# 항상 확인하기
- 학습데이터와 테스트 데이터의 피처개수는 동일해야 함

In [92]:
train_ft.isnull().sum().sum(), test_ft.isnull().sum().sum()

(0, 0)

In [93]:
train_ft.shape, test_ft.shape

((14940, 268), (12225, 268))

# 추출한 피처 저장하기

In [94]:
train_ft.to_csv("train_common_v2.3_1104.csv",index=False)
test_ft.to_csv("test_common_v2.3_1104.csv",index=False)

In [ ]:
DATA_PATH = "C:/Users/user/Desktop/데이터분석/05 Project_Final/Feature_csv"
train_ft = pd.read_csv(f"{DATA_PATH}train_common_v2.0_1101.csv")
test_ft = pd.read_csv(f"{DATA_PATH}test_common_v2.0_1101.csv")

In [166]:
train_ft.shape, test_ft.shape

((14940, 465), (12225, 465))

In [85]:
train_ft.isnull().sum().sum(), test_ft.isnull().sum().sum()

(0, 0)

# 점수 확인

In [95]:
train_ft.drop(columns= "ID", inplace= True)
test_ft.drop(columns= "ID", inplace= True)

In [96]:
cols = train_ft.select_dtypes("object").columns
cols

Index(['주구매지점', '주구매_대분류_수정', '주구매_중분류_수정', '대분류_수정_평균금액최대'], dtype='object')

In [97]:
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler

# 범주형 변수 원핫인코딩
enc = OneHotEncoder(handle_unknown = 'ignore')
enc.fit(train_ft[cols])
tmp = pd.DataFrame(
    enc.transform(train_ft[cols]).toarray(),
    columns = enc.get_feature_names_out()
)
train_ft = pd.concat([train_ft,tmp],axis=1).drop(columns=cols)

tmp = pd.DataFrame(
    enc.transform(test_ft[cols]).toarray(),
    columns = enc.get_feature_names_out()
)
test_ft = pd.concat([test_ft,tmp],axis=1).drop(columns=cols)

# Min-Max Scaling
scaler = StandardScaler()
scaler.fit(train_ft)
train_ft[train_ft.columns] = scaler.transform(train_ft)
test_ft[test_ft.columns] = scaler.transform(test_ft)

# 정답 데이터
target = train_target["target"]

In [98]:
import numpy as np
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold

params = {'n_estimators': 800,
 'learning_rate': 0.04036413044768581,
 'max_depth': 4,
 'min_child_weight': 7,
 'subsample': 0.7505214930635562,
 'colsample_bytree': 0.6290102054237857,
 'gamma': 0.648553153047272}

# F1 매크로 스코어와 모델을 저장할 리스트
scores = []
models = []

# Stratified K-Fold 교차 검증 설정
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 교차 검증 루프
for tri, vai in cv.split(train_ft, target):
    # 학습 데이터와 검증 데이터 분리
    x_train = train_ft.iloc[tri]
    y_train = target.iloc[tri]
    x_valid = train_ft.iloc[vai]
    y_valid = target.iloc[vai]

    # 모델 초기화 및 학습
    model = XGBClassifier(**params)
    model.fit(x_train, y_train, eval_set=[(x_valid, y_valid)], verbose=False)   
    
    # 모델 저장
    models.append(model)

    # 예측 및 F1 매크로 스코어 계산
    pred = model.predict(x_valid)
    score = f1_score(y_valid, pred, average='macro')
    scores.append(score)

# F1 매크로 스코어의 평균 출력
print("Mean F1 Macro Score:", np.mean(scores))


Mean F1 Macro Score: 0.701819076212095


In [99]:
scores

[0.7037877333128504,
 0.700004893139275,
 0.6998877039105895,
 0.7110895279393836,
 0.6943255227583771]

In [93]:

params = {'n_estimators': 800,
 'learning_rate': 0.04036413044768581,
 'max_depth': 4,
 'min_child_weight': 7,
 'subsample': 0.7505214930635562,
 'colsample_bytree': 0.6290102054237857,
 'gamma': 0.648553153047272}

model = XGBClassifier(**params)
model.fit(train_ft, target)
pred = model.predict(test_ft)

In [94]:
submit["target"] = pred

submit.to_csv("6조_v2.1_1104.csv",index=False)

In [ ]:
pd.set_option('display.max_rows', None)
importance = model.feature_importances_  
feature_names = train_ft.columns 
importance_list = []
for i in range(len(importance)):
    importance_list.append([feature_names[i], importance[i]])
pd.DataFrame(importance_list, columns=["feature_names", "importance"]).sort_values(by= "importance", ascending= False)

In [96]:
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=42)   # 특성 선택을 하기 위한 모델
cv = StratifiedKFold(5, shuffle= True, random_state= 42)

from sklearn.feature_selection import SelectFromModel
fs = SelectFromModel(rf)    # 특성선택에 사용하기 위한 모델의 객체 전달
fs.fit(train_ft, target)    # fit : 평균 이상의 중요도를 가진 피처의 컬럼들을 저장
x_train = fs.transform(train_ft)     # transform : 입력 데이터를 저장된 컬럼명의 피처들만 선택해서 넘파이 형태로 반환해줌

scores = cross_val_score(model, x_train, target, cv= cv, scoring= "f1_macro", n_jobs= -1)
scores.mean()

0.7010463996128611

In [98]:
fs.get_feature_names_out()

array(['내점일수', '구매주기', '주말방문비율', '봄_구매비율', '여름_구매비율', '가을_구매비율',
       '겨울_구매비율', '주구매요일', '일별평균구매건수', '아침_구매비율', '점심_구매비율', '저녁_구매비율',
       '아침_구매건수', '점심_구매건수', '저녁_구매건수', '월초_구매비율', '월말_구매비율', '월초_구매건수',
       '월말_구매건수', '주_구매_월', '주_구매시간대', '웨딩성수기_구매비율', '웨딩성수기_구매횟수',
       '1월_구매비율', '2월_구매비율', '3월_구매비율', '4월_구매비율', '5월_구매비율', '6월_구매비율',
       '7월_구매비율', '8월_구매비율', '9월_구매비율', '10월_구매비율', '11월_구매비율',
       '12월_구매비율', '공휴일_구매비율', '여름휴가_구매비율', '연말_구매비율', '1월_구매금액',
       '2월_구매금액', '3월_구매금액', '4월_구매금액', '5월_구매금액', '6월_구매금액', '7월_구매금액',
       '8월_구매금액', '9월_구매금액', '10월_구매금액', '11월_구매금액', '12월_구매금액', '최근구매일',
       '지점다양성_비율', '브랜드코드_nunique', '브랜드다양성_비율', '대분류_수정_아동_구매횟수',
       '대분류_수정_생식품_구매횟수', '대분류_수정_의류_구매횟수', '총구매액', '구매건수', '평균구매액',
       '최대구매액', '최소구매액', '환불금액', '구매금액표준편차', '저가_구매횟수', '중저가_구매횟수',
       '중고가_구매횟수', '고가_구매횟수', '저가_구매비율', '중저가_구매비율', '중고가_구매비율',
       '고가_구매비율', '대분류_수정_생식품_구매금액', '대분류_수정_의류_구매금액', '대분류_수정_아동_구매금액',
       '대분류_수정_가정용품_구매금액', '대분류

In [100]:
from flaml import AutoML
import xgboost 
from xgboost import XGBRFClassifier

auto_ml_ens = AutoML()
params = { "metric" : "macro_f1",
           "task" : "classification",
           "time_budget" : 60*10,
           "seed" : 42,
           "early_stop" : True,
           "ensemble" : {'final_estimator' : XGBRFClassifier() },    # 메타모델이 로지스틱 회귀!
           "estimator_list" : ['catboost', 'lgbm', 'rf', 'xgboost', 'histgb', 'lrl1', 'lrl2', 'kneighbor']  }   # 앙상블에 사용할 모델 지정
auto_ml_ens.fit(train_ft, target, **params)

[flaml.automl.logger: 11-05 14:08:32] {1728} INFO - task = classification
[flaml.automl.logger: 11-05 14:08:32] {1739} INFO - Evaluation method: holdout
[flaml.automl.logger: 11-05 14:08:32] {1838} INFO - Minimizing error metric: 1-macro_f1
[flaml.automl.logger: 11-05 14:08:32] {1955} INFO - List of ML learners in AutoML Run: ['catboost', 'lgbm', 'rf', 'xgboost', 'histgb', 'lrl1', 'lrl2', 'kneighbor']
[flaml.automl.logger: 11-05 14:08:32] {2258} INFO - iteration 0, current learner catboost
[flaml.automl.logger: 11-05 14:08:38] {2393} INFO - Estimated sufficient time budget=56421s. Estimated necessary time budget=89s.
[flaml.automl.logger: 11-05 14:08:38] {2442} INFO -  at 6.5s,	estimator catboost's best error=0.3225,	best estimator catboost's best error=0.3225
[flaml.automl.logger: 11-05 14:08:38] {2258} INFO - iteration 1, current learner lgbm
[flaml.automl.logger: 11-05 14:08:38] {2442} INFO -  at 6.9s,	estimator lgbm's best error=0.6223,	best estimator catboost's best error=0.3225
[

In [104]:
auto_ml_ens.best_loss

0.3003927358629854

In [105]:
pred = auto_ml_ens.predict(test_ft)
pred.mean()

0.3401226993865031

In [107]:
submit["target"] = pred

submit.to_csv("6조_v2.3_1105.csv",index=False)